In [1]:
from typing import Callable

import numpy as np
import pandas as pd

In [2]:
class Perceptron():
    """Object representing perceptron with two inputs.

    Attributes:
        e: A training set.
        w0: Bias weight.
        w1: The weight of the first input.
        w2: The weight of the second input.
        epochs: Number of epochs before stabilisation.
    """
    def __init__(self, e, activation_function = lambda s: 1 if s > 0 else -1):
        '''Initialises Perceptron object.'''
        self.w0 = 0
        self.w1 = 0
        self.w2 = 0
        self.e = e
        self.activation_function = activation_function

    def train(self):
        """Trains perceptron."""
        self.epochs = 1
        stable = False
        while not stable:
            stable = True
            for example in self.e:
                print(example)
                if self.classify(example[1], example[2]) == example[3]:
                    pass
                else:
                    self.w0 += example[3] * example[0]
                    self.w1 += example[3] * example[1]
                    self.w2 += example[3] * example[2]
                    stable = False
            if not stable:
                self.epochs += 1

    def classify(self, x1, x2):
        """Classifies an object."""
        s = (self.w1 * x1) + (self.w2 * x2) + self.w0
        return self.activation_function(s)



## Activation Functions

### Sigmoid Function

#### First Order Derivative

We also need to calculate the first order differential of the function.

$$
f'(S_j) = u_j (1 - u_j)
$$

#### Delta Values

- $\delta_j = (C - u_O) f'(S_O)$ O is the output node.
- $\delta_j = w_{j,O} \delta_O f'(S_j)$ for hidden layer nodes.


In [3]:
class Sigmoid:
    """Represents Sigmoid activation function.
    """
    
    def __init__(self):
        """Initialises a sigmoid object.
        """
        self.vectorised_func = np.vectorize(self.func)
        self.vectorised_der = np.vectorize(self.der)
        
    def func(self, x):
        """Calculates output of the Sigmoid function.
        """
        return 1 / (1 + np.e ** (-x))
    
    def der(self, x):
        """Calculates output of the derivative of the Sigmoid function.
        """
        return self.func(x) * (1 - self.func(x))

### Tanh

#### First Order Derivative

$$
f'(S_j) = 1 - u^2_j
$$

In [4]:
"""Contains definition of a layer of a neural network.
"""

np.random.seed(0)

X = [[1, 2, 3, 2.5],
     [2.0, 5.0, -1.0, 2.0],
     [-1.5, 2.7, 3.3, -0.8]]

class Layer:
    """Layer of a neural network.
    
    Attributes:
        weights: set of weights of the layer.
        biases: set of biases of the layer.
        activation_function: Activation Function of the layer.
        number_of_inputs: Number of inputs coming to the layer.
        number_of_inputs: Number of inputs coming to the layer.
        output: the most recent output of the layer.
    """
    def __init__(self,
                 number_of_inputs: int,
                 number_of_neurons: int,
                 activation_function
                ):
        """Initialises a NeuralNetwork instance.
        """
        self.number_of_neurons = number_of_neurons
        self.activation_function = activation_function
        random_generator = np.random.default_rng(5)
        #NNFS self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        low = -2 / number_of_neurons
        high = 2 / number_of_neurons
        self.weights = random_generator.uniform(
            low=low,
            high=high,
            size=(number_of_inputs, number_of_neurons)
        )
        self.biases = random_generator.uniform(
            low=low,
            high=high,
            size=(1, number_of_neurons)
        )
        self.delta = np.nan
    
    def forward_pass(self, inputs: np.ndarray):
        """Does the forward pass through the layer.
        
        Args:
            inputs: Inputs to the layer.
        """
        self.sum = np.dot(inputs, self.weights) + self.biases
        self.output = self.activation_function.vectorised_func(self.sum)
        
    def update_weights(self, learning_parameter, inputs):
        """Updates the layer's weights.
        
        Args:
            learning_parameter: Learning parameter of the network.
            inputs: Inputs to the layer.
        """
        self.weights = self.weights + learning_parameter * np.dot(inputs, self.delta)

In [5]:
class HiddenLayer(Layer):
    """Hidden Layer of a neural network.
    
    Attributes:
        weights: set of weights of the layer.
        biases: set of biases of the layer.
        activation_function: Activation Function of the layer.
        number_of_inputs: Number of inputs coming to the layer.
        number_of_neurons: Number of neurons in the layer.
        output: the most recent output of the layer.
        delta: Delta value (output of the backward pass) of the layer.
    """
    
    def backward_pass(self, output_weights, output_delta):
        """Does the backward pass through the layer.
        
        Args:
            y: Correct output (label) of the training example.
        """
        self.delta = self.activation_function.vectorised_der(self.sum) * output_delta * output_weights

In [6]:
class OutputLayer(Layer):
    """Single-node output layer of a neural network.
    
    Attributes:
        weights: set of weights of the layer.
        biases: set of biases of the layer.
        activation_function: Activation Function of the layer.
        number_of_inputs: Number of inputs coming to the layer.
        number_of_neurons: Number of neurons in the layer.
        output: the most recent output of the layer.
        delta: Delta value (output of the backward pass) of the layer.
    """
        
    def backward_pass(self, y):
        """Does the backward pass through the layer.
        
        Args:
            y: Correct output (label) of the training example.
        """
        self.delta = (y - self.output) * self.activation_function.vectorised_der(self.sum)

In [7]:
"""Contains NeuralNetwork class definition.

Run after running the Data Preprocessing notebook.
"""

class NeuralNetwork:
    """A neural network with single hidden layer and single node on outputlayer.
    
    Attributes:        
        training_set: Set that instance is to be trained on.
        test_set: Set that instance is to be tested on.
        hidden_nodes: Number of nodes on the hidden layer.
        activation_function: Activation function used for
            the neural network.
        step_size: Step size parameter.
    """
    
    def __init__(
        self,
        training_set: np.ndarray,
        test_set: np.ndarray,
        hidden_nodes: int,
        activation_function,
        step_size: float = 0.1,
     
    ):
        """Initialises a NeuralNetwork instance.
        """
        
        # Input is a matrix where each row is one instance
        self.training_set = training_set
        self.test_set = test_set
        self.step_size = step_size
        self.number_of_inputs = self.training_set.shape[1] - 1
        self.hidden_layer = HiddenLayer(
            self.number_of_inputs,
            hidden_nodes,
            activation_function
        )
        self.output_layer = OutputLayer(
            self.hidden_layer.number_of_neurons,
            1,
            activation_function
        )
    
    
    def train(self):
        """Trains NeuralNetwork instance.
        """
        # Loop through the training set. 
        for training_example in self.training_set:
            # Split individual examples into inputs (item) and label (c).
            item, c = np.hsplit(training_example, [self.training_set.shape[1] - 1])
            print(item)
            print(f"Label: {c}")
            print("===")
            self.forward_pass(item)
            self.backward_pass(c)
            self.update_weights(item)
            
    def forward_pass(self, inputs):
        """Performs forward pass through the network.
        
        Args:
        inputs: Vector of values represneting a training example.
        """
        self.hidden_layer.forward_pass(inputs)
        self.output_layer.forward_pass(self.hidden_layer.output)
                
    
    def backward_pass(self, c):
        """Performs backward pass through the network.
        
        Args:
        c: The label for the training example.
        """
        self.output_layer.backward_pass(c)
        self.hidden_layer.backward_pass(
            self.output_layer.weights,
            self.output_layer.delta
        )
        
    def update_weights(self, inputs):
        """Update weights in the network.
        """
        self.hidden_layer.update_weights(self.step_size, inputs)
        self.output_layer.update_weights(self.step_size, self.hidden_layer.output)
        
        
    def test(self):
        """Tests NeuralNetwork instance.
        """
        pass
    
    def predict(self):
        """Predicts value for given predictor values.
        """
        pass
    
    

In [8]:
# Train.

training_set = pd.read_csv("data/training-set.csv")
training_set = training_set.to_numpy() # Convert to a numpy array.
training_set = training_set[:, 1:] # Get rid of the index column.
training_set

neural_network = NeuralNetwork(
    training_set = training_set,
    test_set = None,
    hidden_nodes = 8,
    activation_function = Sigmoid()
)
neural_network.train()

[0.1286054827175209 0.1488935721812433 0.1630705394190871
 0.1109696376101861 0.1218312019319253 0.1169254318827903
 0.1174053508329126 0.1093817997356863]
Label: [0.1186784156829565]
===


ValueError: shapes (1,8) and (1,1) not aligned: 8 (dim 1) != 1 (dim 0)

## Batch Learning

Batch size may improve efficiency. Showing all sampes at once can cause overfitting. It will be bad at generalsing.

Typical batch size: 32



In [ ]:
inputs = [[1, 2, 3, 2.5],
          [2.0, 5.0, -1.0, 2.0],
          [-1.5, 2.7, 3.3, -0.8]]

weights = [[0.2, 0.8, -0.5, 1.0],
          [0.5, -.91, 0.26, -0.5],
          [-0.26, -.27, 0.17, 0.87]]

biases = [2, 3, 0.5]


weights2 = [[0.1, -0.14, 0.5],
          [-0.5, 0.12, -0.33],
          [-0.44, 0.73, -0.13]]

biases2 = [-1, 2, -0.5]

layer1_outputs = np.dot(inputs, np.array(weights).T) + biases

layer2_outputs = np.dot(layer1_outputs, np.array(weights2).T) + biases2

print(layer2_outputs)

## Feature Data Set

Feature data set is usaully denoted with `X`.

Labels are usually denoted with `y`.

## Activation Functions

Every node on hidden layers and output layer have an activation function.

ReLU

sigmoid func has gradient problem.

## Learning Rate

Gradient of the error function

Too small weights - stuck in local minima

## New Weights

$$
w^*_{i,j} = w_{i,j} + \rho \delta_i u_i
$$